In [1]:
#!/usr/bin/python3

from PIL import Image, ImageDraw, ImageFont
import time
from rgbmatrix import RGBMatrix, RGBMatrixOptions
import asyncio
import math

In [2]:
# Configuration for the matrix

options = RGBMatrixOptions()
options.rows = 64
options.cols = 128
options.chain_length = 2
options.led_rgb_sequence = 'RBG'
options.panel_type = 'FM6126A'
options.hardware_mapping = 'regular'
options.gpio_slowdown = 1
options.pwm_bits = 11
options.pwm_dither_bits = 2
options.pwm_lsb_nanoseconds = 130
options.brightness = 30

matrix = RGBMatrix(options = options)

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [151]:
class Widget():
    created_list=[] #makes Widgets unable to delete, but makes Widget.stop_all() work
    def stop_all():
        for w in Widget.created_list:
            w.stop()
    def running():
        r=[]
        for w in Widget.created_list:
            if w._running: r.append(w)
        return r
    def __init__(self, wigth, height, background_color="black"):
        Widget.created_list.append(self)
        self.size = (wigth, height)
        self.w, self.h = self.size
        self.background_color = background_color
        self._running=False
        
        self.image = Image.new("RGB", self.size, background_color)
        self.draw = ImageDraw.Draw(self.image)
        
    def clear(self, color = None):
        color = self.background_color if color == None else color 
        self.draw.rectangle((0,0)+self.size, fill = color)
    def place_once(self, x, y):
        matrix.SetImage(self.image, x, y)
    def update_frame(self):
        pass
        #to be defined in subclasses individually
    async def _start(self, x, y, refresh_rate = 5):
        self._running = True
        while self._running:
            self.place_once(x, y)
            rr=refresh_rate
            await asyncio.sleep(rr - time.time() % rr)
            self.update_frame()
    def start(self, *args, **kwargs):
        assert(not self._running)
        
        # 1 widget object -> 1 widget copy on matrix
        # self._start() still can be started manually, when you need it
        self.task = asyncio.create_task(self._start(*args, **kwargs))
    def stop(self):
        #self._running = False
        self.task.cancel()
        #TODO wait for task to finish

In [130]:
class Test_W(Widget):
    colors = ["yellow" ,"red", "green", "blue", "white"]
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.n = 0
    def update_frame(self):
        y = self.n % self.h
        color = self.colors[self.n//self.h]
        self.draw.line((0, y,self.w, y), fill=color)
        self.n += 1
        self.n %= 64*len(self.colors)

async def run_test():
    example = Test_W(256, 64)
    example.start(0,0, refresh_rate=0.001)
    await asyncio.sleep(3)
    example.stop()
    matrix.Clear()
asyncio.create_task(run_test())

<Task pending coro=<run_test() running at <ipython-input-130-3383db89410d>:13>>

In [15]:
DEFAULT_FONT = 'fonts/terminess.ttf'
class Time_W(Widget):
    def __init__(self, *args,
                 timezone=+3, t_format="%p %I:%M:%S",
                 font_name=DEFAULT_FONT, font_color="#ffffff",
                 **kwargs):
        super().__init__(*args, **kwargs)
        self.timezone = timezone
        self.t_format = t_format
        self.font_obj = ImageFont.truetype(font_name, self.h)
        self.font_color = font_color
    def update_frame(self):
        self.clear()
        t = time.time() + 3600*self.timezone
        t = time.gmtime(t)
        t_str = time.strftime(self.t_format, t)
        self.draw.text((0,0), t_str, fill=self.font_color, font=self.font_obj)

In [6]:
class Time_Arrows_W(Widget):
    def __init__(self, *args, timezone=+3, **kwargs):
        super().__init__(*args, **kwargs)
        self.timezone = timezone
    def update_frame(self):
        self.clear()
        t = time.time()
        r = min(self.w, self.h)/2
        for i in range(12):
            x1 = r + r*math.cos(math.pi*i/6)
            y1 = r + r*math.sin(math.pi*i/6)
            x2 = r + 0.93*r*math.cos(math.pi*i/6)
            y2 = r + 0.93*r*math.sin(math.pi*i/6)
            fill="#ffffff" if i%3==0 else "#990000"
            self.draw.line((x1,y1,x2,y2), fill=fill)
        
        hx =   0.6*r*math.sin(math.pi*( (t/3600)%24+self.timezone )/6)
        hy = - 0.6*r*math.cos(math.pi*( (t/3600)%24+self.timezone )/6)
        self.draw.line((r,r,r+hx,r+hy), fill="#ffffff")
        
        mx =   0.9*r*math.sin(math.pi*( (t/60)%60 )/30)
        my = - 0.9*r*math.cos(math.pi*( (t/60)%60 )/30)
        self.draw.line((r,r,r+mx,r+my), fill="#ffffff")
        
        sx =   r*math.sin(math.pi*(t%60)/30)
        sy = - r*math.cos(math.pi*(t%60)/30)
        self.draw.line((r+sx,r+sy,r+0.95*sx,r+0.95*sy), fill="#ffffff")

In [ ]:
class Weather_W(Widget):
    def __init__(self, *args,
                 location=+3, w_format="",
                 font_name=DEFAULT_FONT, font_color="#ffffff",
                 **kwargs):
        super().__init__(*args, **kwargs)
        self.location = location
        self.w_format = w_format
        self.font_obj = ImageFont.truetype(font_name, self.h)
        self.font_color = font_color
    def update_frame(self):
        self.clear()
        w = 
        t_str = time.strftime(self.t_format, t)
        self.draw.text((0,0), t_str, fill=self.font_color, font=self.font_obj)

In [155]:
#Widget.stop_all()
#matrix.Clear()
time_bashk = Time_W(140, 27, timezone=+5, t_format="%p%I:%M:%S %d")
time_msk   = Time_W(137, 18, timezone=+3, t_format="   %I:%M %p MSK", font_color='#eeeeee')
time_cdt   = Time_W(137, 18, timezone=-5, t_format="   %I:%M %p CDT", font_color='#eeeeee')

time_bashk.start(0, -3, refresh_rate = 1)
time_msk.start(3, 20, refresh_rate = 1)
time_cdt.start(3, 20+16, refresh_rate = 1)

#analog=Time_Arrows_W(50,50,timezone = +5)
#analog.start(144, 0, refresh_rate = 1)